In [ ]:
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

In [3]:
!cat models/20192009_2/train.csv | wc -l

47800


In [4]:
!ls models/20192009_2/

train-data-10_1.csv  train-data-1_5.csv   train-data-31_6.csv
train-data-10_4.csv  train-data-1_6.csv   train-data-33_5.csv
train-data-13_1.csv  train-data-1_7.csv   train-data-34_4.csv
train-data-13_5.csv  train-data-1_8.csv   train-data-3_1.csv
train-data-15_4.csv  train-data-1_9.csv   train-data-40_2.csv
train-data-17_3.csv  train-data-20_5.csv  train-data-40_4.csv
train-data-18_2.csv  train-data-21_3.csv  train-data-5_5.csv
train-data-19_2.csv  train-data-21_4.csv  train-data-6_1.csv
train-data-1_3.csv   train-data-2_4.csv   train.csv
train-data-1_4.csv   train-data-30_3.csv


In [5]:
features = ["f" + str(i) for i in range(0,117)]
label = ["action"]

headers = features + label

df = pd.read_csv('models/20192009_2/train.csv', 
                 sep = ',', 
                 header = None,
                 names = headers)
df

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f108,f109,f110,f111,f112,f113,f114,f115,f116,action
0,0,0,0,0,0,0,0,0,0,0,...,19,-63,-2,6,677,-178,653,410,-63,0
1,-8,232,-3,1,677,18,677,214,-87,1,...,16,-60,-2,6,671,-176,647,408,-57,4
2,-8,232,-3,1,676,21,676,211,-86,1,...,13,-57,-2,6,664,-174,641,406,-51,5
3,-8,232,-3,1,675,24,675,208,-85,1,...,10,-54,-2,6,656,-164,635,404,-45,1
4,8,240,-3,1,675,35,674,205,-84,1,...,7,-51,-2,6,650,-162,629,402,-39,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47795,0,65,-2,7,65,-42,50,107,540,0,...,165,419,-2,1,575,-96,567,161,23,4
47796,0,65,-2,7,59,-40,43,105,547,0,...,166,424,-2,1,572,-86,566,159,24,1
47797,8,73,-2,7,47,-30,36,103,554,0,...,167,429,-2,1,571,-84,565,157,25,2
47798,8,73,-2,7,40,-28,29,101,561,0,...,168,434,-2,1,570,-82,564,155,26,3


In [6]:
print("headers length: {}, df lenght: {}".format(len(headers), len(df)))

headers length: 118, df lenght: 47800


In [7]:
df["f0"].value_counts()

-8    19446
 8    18258
 0    10096
Name: f0, dtype: int64

In [8]:
print("different actions count \n{}".format(df.drop_duplicates()['action'].value_counts()))

different actions count 
5    9334
0    9151
1    8737
4    7959
2    7846
3    4773
Name: action, dtype: int64


In [9]:
print("no of states without duplicates: {}".format(len(df.drop_duplicates())))

no of states without duplicates: 47800


In [10]:
y_train_all = df["action"]
X_train_all = df.drop(columns="action")

In [11]:
print("no of states without duplicates and action column: {}".format(len(X_train_all.drop_duplicates())))

no of states without duplicates and action column: 47800


In [ ]:
# import pandas_profiling
# pandas_profiling.ProfileReport(df)

In [ ]:
# X_train_all[["f0", "f1"]]

In [12]:
def evaluate_model(xtrain, ytrain, xtest, ytest, model):
    y_pred_test = model.predict(xtest)
    y_pred_train = model.predict(xtrain)
    test_accuracy = accuracy_score(ytest, y_pred_test)
    train_accuracy = accuracy_score(ytrain, y_pred_train)
    print('Training accuracy: {0:.2f}, evaluation accuracy: {1:.2f}'.format(train_accuracy, test_accuracy))


Basic logisting regresion

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, test_size=0.2)

classifier = LogisticRegression().fit(X_train, y_train)

/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/envs/letsplay/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [14]:
evaluate_model(X_train, y_train, X_test, y_test, classifier)

Training accuracy: 0.58, evaluation accuracy: 0.57


In [15]:
pickle.dump(classifier, open("ai_model_logit.pkl","wb"))

xgboost

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_train_all, y_train_all, 
                                                    test_size=0.2, random_state=7)

xgb_model = XGBClassifier(n_estimators=500).fit(X_train, y_train,eval_set = [[X_test, y_test]], verbose = 0)

In [17]:
evaluate_model(X_train, y_train, X_test, y_test, xgb_model)

Training accuracy: 0.76, evaluation accuracy: 0.66


In [ ]:
xgb_model = XGBClassifier(n_estimators=500).fit(X_train_all, y_train_all)

In [18]:
pickle.dump(xgb_model, open("ai_model_xgb.pkl","wb"))

random forest model

In [21]:
forest_model = RandomForestClassifier(random_state=1, n_estimators=500)
forest_model.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [22]:
evaluate_model(X_train, y_train, X_test, y_test, forest_model)

Training accuracy: 1.00, evaluation accuracy: 0.52


In [ ]:
# min_max_scaler = MinMaxScaler()
# np_scaled = min_max_scaler.fit_transform(df)
# df_normalized = pd.DataFrame(np_scaled)
# df_normalized.columns = feature_cols + ['Class']

In [ ]:
# def make_crosses(X_train, X_validation, degree=2):
#   from sklearn.preprocessing import PolynomialFeatures
#   poly = PolynomialFeatures(degree=degree)
#   X_train_poly = poly.fit_transform(X_train)
#   X_validation_poly = poly.transform(X_validation)
#   return X_train_poly, X_validation_poly

In [23]:
mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(24, 24), random_state=1, max_iter=200000)

mlp_model.fit(X_train_all, y_train_all)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(24, 24), learning_rate='constant',
              learning_rate_init=0.001, max_iter=200000, momentum=0.9,
              n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
              random_state=1, shuffle=True, solver='lbfgs', tol=0.0001,
              validation_fraction=0.1, verbose=False, warm_start=False)

In [24]:
evaluate_model(X_train, y_train, X_test, y_test, mlp_model)

Training accuracy: 0.17, evaluation accuracy: 0.18


Save models to pkl

In [25]:
pickle.dump(forest_model, open("ai_model_forest.pkl","wb"))

In [26]:
pickle.dump(mlp_model, open("ai_model_mlp.pkl","wb"))

In [ ]:
def train_on_all_data(xtrain_all, ytrain_all):
    models = dict()
    X_train, X_test, y_train, y_test = train_test_split(xtrain_all, ytrain_all, 
                                                    test_size=0.2, random_state=7)

    xgb_model = XGBClassifier(n_estimators=1000).fit(X_train, y_train,eval_set = [[X_test, y_test]])
    models['xgb', xgb_model]
    
    mlp_model = MLPClassifier(solver='lbfgs', alpha=1e-5, 
                    hidden_layer_sizes=(28, 32, 5), random_state=1)

    mlp_model.fit(xtrain_all, ytrain_all)
    models['mlp', mlp_model]
        
    forest_model = RandomForestClassifier(random_state=1, n_estimators=1000)
    forest_model.fit(xtrain_all, ytrain_all)
    models['forest', forest_model]
    
    return models

trained_models = train_on_all_data(X_train_all, y_train_all)
for key in trained_models.keys:
    pickle.dump(trained_models[key], open("ai_model_"+ key +".pkl","wb"))

In [ ]:
!ls

In [ ]:
# model_p = pickle.load(open("ai_model.pkl","rb"))
game_state_4f = (np.array([8,280,630,-1,1,674,-3,7,667,-1,2,649,-3,2,662,0,3,662,-1,4,655,-1,5,608,2,7,1,8,288,630,-1,1,672,-3,7,668,-1,2,650,-3,2,661,0,3,657,-1,4,649,-1,5,602,2,7,1,8,296,629,-1,1,670,-3,7,669,-1,2,651,-3,2,660,0,3,651,-1,4,643,-1,5,596,2,7,1,8,296,628,-1,1,665,-3,7,667,-1,2,650,-3,2,658,0,3,647,-1,4,638,-1,5,589,2,7])
                 .reshape(1, -1))



game_state_1f = (np.array([25, -410, 308, 5, 1, -343, 483, 1, 2, -417, 592, 5, 2, -184, 589, 1, 2, -222, 275, 7, 2, -93, 143, 3, -2, -128, 549, 1, -3, -121, 399, 6, -3]).reshape(1, -1))

xgb_model.predict(game_state_1f)